In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from collections import Counter
from imblearn.over_sampling import RandomOverSampler

from bayes_opt import BayesianOptimization

import keras
import tensorflow
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Embedding, concatenate, Lambda, Bidirectional, LSTM, LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras import metrics
from keras.losses import BinaryCrossentropy
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model, model_selection

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

from matplotlib import pyplot as plt

from sklearn import model_selection
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier

from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTEENN
from collections import Counter

In [10]:
from imblearn.over_sampling import RandomOverSampler

In [2]:
X_train = pd.read_csv('../datasets/word2vec/train_data_imputed_FINAL.csv')
X_test = pd.read_csv('../datasets/word2vec/test_data_imputed_FINAL.csv')
y_train = pd.read_csv('../datasets/word2vec/y_train_FINAL.csv').iloc[:,1:]
y_test = pd.read_csv('../datasets/word2vec/y_test_FINAL.csv').iloc[:,1:]

In [17]:
print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')

X_train: (11516, 263)
y_train: (11516, 1)
X_test: (2880, 1, 263)
y_test: (2880, 1)


## Oversampling 

In [8]:
original_counter = Counter(y_train.loc[:,'fraudulent'])
print("Before oversampling:", original_counter)

Before oversampling: Counter({0: 11293, 1: 223})


In [11]:
oversampler = RandomOverSampler(sampling_strategy = 1, random_state=42)
X_train_over, y_train_over = oversampler.fit_resample(X_train, y_train)

In [12]:
after_counter = Counter(y_train_over.loc[:,'fraudulent'])
print("After oversampling:", after_counter)

After oversampling: Counter({0: 11293, 1: 11293})


In [13]:
X_train_over = X_train_over.values
X_train_over = X_train_over.reshape(X_train_over.shape[0], 1, X_train_over.shape[1])

In [14]:
X_test = X_test.values
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Basic NN with Bidirectional LSTM

In [ ]:
# for reproducibility 
np.random.seed(42)
tensorflow.random.set_seed(42)

model1 = Sequential()
model1.add(Bidirectional(LSTM(64, return_sequences=False))) 
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])


In [ ]:
model1.fit(X_train_over, y_train_over, epochs=20, batch_size = 64)

Epoch 1/20
353/353 [==============================] - 6s 4ms/step - loss: 0.3385 - accuracy: 0.8498 - precision: 0.8275 - recall: 0.8839 - auc: 0.9353
Epoch 2/20
353/353 [==============================] - 1s 4ms/step - loss: 0.1360 - accuracy: 0.9569 - precision: 0.9385 - recall: 0.9778 - auc: 0.9885
Epoch 3/20
353/353 [==============================] - 1s 4ms/step - loss: 0.0863 - accuracy: 0.9740 - precision: 0.9585 - recall: 0.9909 - auc: 0.9949
Epoch 4/20
353/353 [==============================] - 1s 4ms/step - loss: 0.0607 - accuracy: 0.9835 - precision: 0.9714 - recall: 0.9964 - auc: 0.9972
Epoch 5/20
353/353 [==============================] - 2s 4ms/step - loss: 0.0415 - accuracy: 0.9891 - precision: 0.9803 - recall: 0.9981 - auc: 0.9988
Epoch 6/20
353/353 [==============================] - 1s 4ms/step - loss: 0.0293 - accuracy: 0.9924 - precision: 0.9859 - recall: 0.9991 - auc: 0.9995
Epoch 7/20
353/353 [==============================] - 1s 4ms/step - loss: 0.0236 - accuracy: 0

In [ ]:
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 128)              167936    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 168,065
Trainable params: 168,065
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
train_results = model1.evaluate(X_train_over, y_train_over)
precision = train_results[2]
recall = train_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Train Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {train_results[1]}\nAUROC:{train_results[4]}')

706/706 [==============================] - 1s 2ms/step - loss: 0.0017 - accuracy: 0.9996 - precision: 0.9991 - recall: 1.0000 - auc: 1.0000
Train Results
Precision: 0.9991152882575989
Recall: 1.0
F1 Score: 0.9995574483634847
Accuracy: 0.9995572566986084
AUROC:1.0000001192092896


In [ ]:
test_results = model1.evaluate(X_test, y_test)
precision = test_results[2]
recall = test_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Test Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {test_results[1]}\nAUROC:{test_results[4]}')

90/90 [==============================] - 0s 2ms/step - loss: 0.0347 - accuracy: 0.9899 - precision: 0.6901 - recall: 0.8750 - auc: 0.9782
Test Results
Precision: 0.6901408433914185
Recall: 0.875
F1 Score: 0.7716535422575659
Accuracy: 0.9899305701255798
AUROC:0.9782476425170898


Add hidden layers

In [ ]:
# for reproducibility 
np.random.seed(42)
tensorflow.random.set_seed(42)

model2 = Sequential()
model2.add(Bidirectional(LSTM(64, return_sequences=False))) #input_shape = (batch_size (1st dimension -> num_rows --> excluded), input_size (2nd dimension -> cols),input_dim (2-1))
model2.add(Dropout(0.2))
model2.add(Dense(16, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(8, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])

In [ ]:
model2.fit(X_train_over, y_train_over, epochs=20, batch_size = 64)

Epoch 1/20
353/353 [==============================] - 6s 5ms/step - loss: 0.3725 - accuracy: 0.8221 - precision_1: 0.7994 - recall_1: 0.8600 - auc_1: 0.9142
Epoch 2/20
353/353 [==============================] - 2s 6ms/step - loss: 0.1245 - accuracy: 0.9580 - precision_1: 0.9434 - recall_1: 0.9745 - auc_1: 0.9869
Epoch 3/20
353/353 [==============================] - 3s 8ms/step - loss: 0.0767 - accuracy: 0.9748 - precision_1: 0.9652 - recall_1: 0.9850 - auc_1: 0.9943
Epoch 4/20
353/353 [==============================] - 2s 6ms/step - loss: 0.0436 - accuracy: 0.9875 - precision_1: 0.9818 - recall_1: 0.9934 - auc_1: 0.9977
Epoch 5/20
353/353 [==============================] - 2s 4ms/step - loss: 0.0303 - accuracy: 0.9907 - precision_1: 0.9871 - recall_1: 0.9943 - auc_1: 0.9988
Epoch 6/20
353/353 [==============================] - 2s 5ms/step - loss: 0.0241 - accuracy: 0.9924 - precision_1: 0.9897 - recall_1: 0.9951 - auc_1: 0.9991
Epoch 7/20
353/353 [==============================] - 2s 4

In [ ]:
print(model2.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 128)              167936    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                2064      
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                      

In [ ]:
train_results = model2.evaluate(X_train_over, y_train_over)
precision = train_results[2]
recall = train_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Train Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {train_results[1]}\nAUROC:{train_results[4]}')

706/706 [==============================] - 2s 2ms/step - loss: 9.7658e-04 - accuracy: 0.9996 - precision_1: 0.9992 - recall_1: 1.0000 - auc_1: 1.0000
Train Results
Precision: 0.9992036819458008
Recall: 1.0
F1 Score: 0.999601682379144
Accuracy: 0.9996015429496765
AUROC:1.0


In [ ]:
test_results = model2.evaluate(X_test, y_test)
precision = test_results[2]
recall = test_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Test Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {test_results[1]}\nAUROC:{test_results[4]}')

90/90 [==============================] - 0s 2ms/step - loss: 0.0465 - accuracy: 0.9899 - precision_1: 0.7077 - recall_1: 0.8214 - auc_1: 0.9435
Test Results
Precision: 0.7076923251152039
Recall: 0.8214285969734192
F1 Score: 0.7603305995110141
Accuracy: 0.9899305701255798
AUROC:0.943466067314148


Add more hidden layers

In [ ]:
# for reproducibility 
np.random.seed(42)
tensorflow.random.set_seed(42)

model3 = Sequential()
model3.add(Bidirectional(LSTM(64, return_sequences=False))) 
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(32, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(16, activation='relu'))
model3.add(Dense(8, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])

In [ ]:
model3.fit(X_train_over, y_train_over, epochs=20, batch_size = 64)

Epoch 1/20
353/353 [==============================] - 7s 8ms/step - loss: 0.2953 - accuracy: 0.8616 - precision_2: 0.8279 - recall_2: 0.9130 - auc_2: 0.9440
Epoch 2/20
353/353 [==============================] - 2s 5ms/step - loss: 0.0782 - accuracy: 0.9749 - precision_2: 0.9646 - recall_2: 0.9861 - auc_2: 0.9934
Epoch 3/20
353/353 [==============================] - 1s 4ms/step - loss: 0.0505 - accuracy: 0.9839 - precision_2: 0.9785 - recall_2: 0.9895 - auc_2: 0.9970
Epoch 4/20
353/353 [==============================] - 2s 6ms/step - loss: 0.0208 - accuracy: 0.9942 - precision_2: 0.9915 - recall_2: 0.9969 - auc_2: 0.9991
Epoch 5/20
353/353 [==============================] - 3s 7ms/step - loss: 0.0155 - accuracy: 0.9960 - precision_2: 0.9944 - recall_2: 0.9977 - auc_2: 0.9994
Epoch 6/20
353/353 [==============================] - 2s 5ms/step - loss: 0.0179 - accuracy: 0.9948 - precision_2: 0.9929 - recall_2: 0.9967 - auc_2: 0.9991
Epoch 7/20
353/353 [==============================] - 2s 4

In [ ]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 128)              167936    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                      

In [ ]:
train_results = model3.evaluate(X_train_over, y_train_over)
precision = train_results[2]
recall = train_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Train Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {train_results[1]}\nAUROC:{train_results[4]}')

706/706 [==============================] - 2s 2ms/step - loss: 1.4743e-06 - accuracy: 1.0000 - precision_2: 1.0000 - recall_2: 1.0000 - auc_2: 1.0000
Train Results
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0
AUROC:1.0


In [ ]:
test_results = model3.evaluate(X_test, y_test)
precision = test_results[2]
recall = test_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Test Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {test_results[1]}\nAUROC:{test_results[4]}')

90/90 [==============================] - 0s 2ms/step - loss: 0.0736 - accuracy: 0.9906 - precision_2: 0.7843 - recall_2: 0.7143 - auc_2: 0.8737
Test Results
Precision: 0.7843137383460999
Recall: 0.7142857313156128
F1 Score: 0.747663566572446
Accuracy: 0.9906250238418579
AUROC:0.8737004995346069


#Hyperparameter tuning (Round 1)

Best test F1 score: 0.826

In [ ]:
def bidirectional_lstm_tuner(batch_size, neurons, activation, optimizer, dropout, dropout_rate, learning_rate, epochs, layers1):

    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                  'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                  'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', 'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)

    def create_model():
      # for reproducibility 
      np.random.seed(42)
      tensorflow.random.set_seed(42)

      tuned_model = Sequential()
      tuned_model.add(Bidirectional(LSTM(neurons, return_sequences=False)))  
      tuned_model.add(Dropout(dropout_rate, seed=42))

      for i in range(layers1): #varying first set of hidden layers
        tuned_model.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
          tuned_model.add(Dropout(dropout_rate, seed=42))

      tuned_model.add(Dense(1, activation='sigmoid')) #output layer
      # tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])
      tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall()])
      return tuned_model

    es = EarlyStopping(monitor="loss", patience=3)
    mc = ModelCheckpoint('Best_tuned_bidirectional_LSTM_.h5', monitor='val_loss',mode='min',verbose=2, save_best_only=True)

    tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0) 
    score = cross_val_score(tune_rnn, X_train_over, y_train_over, fit_params={'callbacks':[es]}, error_score='raise', scoring='f1').mean() #5 fold cross validation default
    return score

In [ ]:
params_bidirectional_lstm ={
    'neurons': (10, 64),
    'activation':(0, 8),
    'optimizer':(0,5),
    'learning_rate':(0.0001, 0.001),
    'batch_size':(32, 128),
    'epochs':(15, 30),
    'layers1':(0,3),
    'dropout':(0, 1),
    'dropout_rate':(0, 0.5)
}

In [ ]:
# Run Bayesian Optimization
bidirectional_lstm_bo = BayesianOptimization(bidirectional_lstm_tuner, params_bidirectional_lstm, random_state=42)
bidirectional_lstm_bo.maximize(init_points=20, n_iter=5)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 2ms/step
| 1         | 0.1172    | 2.996     | 123.3     | 0.732     | 0.2993    | 17.34     | 0.468     | 0.0001523 | 56.77     | 3.006     |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 2ms/step
| 2         | 0.1571    | 5.665     | 33.98     | 0.9699    | 0.4162    | 18.19     | 0.5455    | 0.0002651 | 26.43     | 2.624     |


...

142/142 [==============================] - 1s 2ms/step
| 23        | 0.1451    | 0.0       | 64.07     | 0.0       | 0.0       | 20.19     | 3.0       | 0.0001    | 55.11     | 0.0       |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 2ms/step
| 24        | 0.945     | 5.168     | 60.06     | 0.2805    | 0.1314    | 23.17     | 0.0       | 0.001     | 51.62     | 2.04      |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 2ms/step
| 25        | 0.9319    | 7.126     | 62.24     | 0.5395    | 0.1362    | 20.24     | 0.0       | 0.001     | 54.8      | 1.86      |


In [ ]:
bidirectional_lstm_bo.max['params']

{'activation': 2.8540266135487142,
 'batch_size': 58.96971292998855,
 'dropout': 0.5426960831582485,
 'dropout_rate': 0.07046211248738132,
 'epochs': 27.032954711310595,
 'layers1': 0.22365193103931247,
 'learning_rate': 0.0009881982429404655,
 'neurons': 51.7012175420195,
 'optimizer': 0.993578407670862}

In [ ]:
def create_model_best_params(params):
  learning_rate = params['learning_rate']
  optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax']
  optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),}
  activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', 'relu']
  neurons = round(params['neurons'])
  activation = activationL[round(params['activation'])]
  optimizer = optimizerD[optimizerL[round(params['optimizer'])]]
  batch_size = round(params['batch_size'])
  epochs = round(params['epochs'])
  layers1 = round(params['layers1'])
  dropout = params['dropout']
  dropout_rate = params['dropout_rate']

  # for reproducibility 
  np.random.seed(42)
  tensorflow.random.set_seed(42)
  tuned_model = Sequential()
  tuned_model.add(Bidirectional(LSTM(neurons, return_sequences=False)))  
  tuned_model.add(Dropout(dropout_rate, seed=42))

  for i in range(layers1): #varying first set of hidden layers
    tuned_model.add(Dense(neurons, activation=activation))
    if dropout > 0.5:
      tuned_model.add(Dropout(dropout_rate, seed=42))

  tuned_model.add(Dense(1, activation='sigmoid')) #output layer
  tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])
  return tuned_model

In [ ]:
best_params = {'activation': 2.8540266135487142,
              'batch_size': 58.96971292998855,
              'dropout': 0.5426960831582485,
              'dropout_rate': 0.07046211248738132,
              'epochs': 27.032954711310595,
              'layers1': 0.22365193103931247,
              'learning_rate': 0.0009881982429404655,
              'neurons': 51.7012175420195,
              'optimizer': 0.993578407670862}
best_model = create_model_best_params(best_params)
best_model.fit(X_train_over, y_train_over, epochs=round(best_params['epochs']), batch_size = round(best_params['batch_size']))

Epoch 1/27
383/383 [==============================] - 11s 9ms/step - loss: 0.3353 - accuracy: 0.8549 - precision: 0.8383 - recall: 0.8795 - auc: 0.9375
Epoch 2/27
383/383 [==============================] - 3s 8ms/step - loss: 0.1286 - accuracy: 0.9605 - precision: 0.9418 - recall: 0.9816 - auc: 0.9897
Epoch 3/27
383/383 [==============================] - 3s 7ms/step - loss: 0.0811 - accuracy: 0.9763 - precision: 0.9607 - recall: 0.9933 - auc: 0.9955
Epoch 4/27
383/383 [==============================] - 3s 7ms/step - loss: 0.0534 - accuracy: 0.9857 - precision: 0.9741 - recall: 0.9979 - auc: 0.9982
Epoch 5/27
383/383 [==============================] - 3s 9ms/step - loss: 0.0356 - accuracy: 0.9918 - precision: 0.9846 - recall: 0.9992 - auc: 0.9991
Epoch 6/27
383/383 [==============================] - 4s 11ms/step - loss: 0.0251 - accuracy: 0.9943 - precision: 0.9890 - recall: 0.9998 - auc: 0.9997
Epoch 7/27
383/383 [==============================] - 3s 7ms/step - loss: 0.0188 - accuracy:

In [ ]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 104)              131456    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 104)               0         
                                                                 
 dense (Dense)               (None, 1)                 105       
                                                                 
Total params: 131,561
Trainable params: 131,561
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_results = best_model.evaluate(X_train_over, y_train_over)
precision = train_results[2]
recall = train_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Train Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {train_results[1]}\nAUROC:{train_results[4]}')

706/706 [==============================] - 4s 3ms/step - loss: 8.3697e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000
Train Results
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0
AUROC:1.0


In [ ]:
test_results = best_model.evaluate(X_test, y_test)
precision = test_results[2]
recall = test_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Test Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {test_results[1]}\nAUROC:{test_results[4]}')

90/90 [==============================] - 0s 4ms/step - loss: 0.0336 - accuracy: 0.9934 - precision: 0.8491 - recall: 0.8036 - auc: 0.9532
Test Results
Precision: 0.849056601524353
Recall: 0.8035714030265808
F1 Score: 0.8256880588457645
Accuracy: 0.9934027791023254
AUROC:0.9531503915786743


#Hyperparameter tuning (Round 2)

Best test F1 score: 0.739

In [ ]:
def bidirectional_lstm_tuner(batch_size, neurons, activation, optimizer, dropout_rate, learning_rate, epochs, layers1):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                  'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                  'Adagrad':Adagrad(learning_rate=learning_rate),}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)

    def create_model():
      # for reproducibility 
      np.random.seed(42)
      tensorflow.random.set_seed(42)

      tuned_model = Sequential()
      tuned_model.add(Bidirectional(LSTM(neurons, return_sequences=False)))  
      tuned_model.add(Dropout(dropout_rate, seed=42))

      for i in range(layers1): #varying first set of hidden layers
        tuned_model.add(Dense(neurons, activation=activation))
        tuned_model.add(Dropout(dropout_rate, seed=42))

      tuned_model.add(Dense(1, activation='sigmoid')) #output layer
      # tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])
      tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall()])
      return tuned_model

    es = EarlyStopping(monitor="loss", patience=3)
    mc = ModelCheckpoint('Best_tuned_bidirectional_LSTM_.h5', monitor='val_loss',mode='min',verbose=2, save_best_only=True)

    tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0) 
    score = cross_val_score(tune_rnn, X_train_over, y_train_over, fit_params={'callbacks':[es]}, error_score='raise', scoring='f1').mean() #5 fold cross validation default
    return score

In [ ]:
params_bidirectional_lstm ={
    'neurons': (10, 64),
    'activation':(0, 7),
    'optimizer':(0, 4),
    'learning_rate':(0.0001, 0.001),
    'batch_size':(32, 128),
    'epochs':(15, 30),
    'layers1':(0,3),
    'dropout_rate':(0, 0.4)
}

In [ ]:
# Run Bayesian Optimization
bidirectional_lstm_bo = BayesianOptimization(bidirectional_lstm_tuner, params_bidirectional_lstm, random_state=42)
bidirectional_lstm_bo.maximize(init_points=20, n_iter=5)

|   iter    |  target   | activa... | batch_... | dropou... |  epochs   |  layers1  | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 2ms/step
| 1         | 0.4619    | 2.622     | 123.3     | 0.2928    | 23.98     | 0.4681    | 0.0002404 | 13.14     | 3.465     |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 2ms/step
| 2         | 0.9354    | 4.208     | 99.97     | 0.008234  | 29.55     | 2.497     | 0.0002911 | 19.82     | 0.7336    |


...

142/142 [==============================] - 1s 2ms/step
| 23        | 0.5147    | 4.233     | 81.79     | 0.1885    | 25.23     | 1.856     | 0.0008313 | 51.19     | 0.4253    |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 2ms/step
| 24        | 0.1943    | 6.81      | 75.42     | 0.2822    | 26.81     | 0.9503    | 0.0007072 | 52.41     | 0.3749    |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 2ms/step
| 25        | 0.1416    | 1.831     | 110.7     | 0.3918    | 17.27     | 2.53      | 0.0007728 | 45.19     | 3.627     |


In [ ]:
bidirectional_lstm_bo.max['params']

{'activation': 6.210489198034286,
 'batch_size': 77.33263281554713,
 'dropout_rate': 0.04783769837532068,
 'epochs': 25.698671808344926,
 'layers1': 2.2823551458506923,
 'learning_rate': 0.0006051494778125466,
 'neurons': 51.63222771754629,
 'optimizer': 1.975182385457563}

In [ ]:
def create_model_best_params(params):
  learning_rate = params['learning_rate']
  optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad']
  optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                'Adagrad':Adagrad(learning_rate=learning_rate),}
  activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'relu']
  neurons = round(params['neurons'])
  activation = activationL[round(params['activation'])]
  optimizer = optimizerD[optimizerL[round(params['optimizer'])]]
  batch_size = round(params['batch_size'])
  epochs = round(params['epochs'])
  layers1 = round(params['layers1'])
  dropout_rate = params['dropout_rate']

  # for reproducibility 
  np.random.seed(42)
  tensorflow.random.set_seed(42)
  tuned_model = Sequential()
  tuned_model.add(Bidirectional(LSTM(neurons, return_sequences=False)))  
  tuned_model.add(Dropout(dropout_rate, seed=42))

  for i in range(layers1): #varying first set of hidden layers
    tuned_model.add(Dense(neurons, activation=activation))
    tuned_model.add(Dropout(dropout_rate, seed=42))

  tuned_model.add(Dense(1, activation='sigmoid')) #output layer
  tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])
  return tuned_model

In [ ]:
best_params = {'activation': 6.210489198034286,
              'batch_size': 77.33263281554713,
              'dropout_rate': 0.04783769837532068,
              'epochs': 25.698671808344926,
              'layers1': 2.2823551458506923,
              'learning_rate': 0.0006051494778125466,
              'neurons': 51.63222771754629,
              'optimizer': 1.975182385457563}
best_model2 = create_model_best_params(best_params)
best_model2.fit(X_train_over, y_train_over, epochs=round(best_params['epochs']), batch_size = round(best_params['batch_size']))

Epoch 1/26
294/294 [==============================] - 8s 8ms/step - loss: 0.3524 - accuracy: 0.8367 - precision_252: 0.8186 - recall_252: 0.8650 - auc_2: 0.9231
Epoch 2/26
294/294 [==============================] - 3s 11ms/step - loss: 0.1588 - accuracy: 0.9392 - precision_252: 0.9255 - recall_252: 0.9554 - auc_2: 0.9826
Epoch 3/26
294/294 [==============================] - 2s 8ms/step - loss: 0.1106 - accuracy: 0.9604 - precision_252: 0.9493 - recall_252: 0.9727 - auc_2: 0.9907
Epoch 4/26
294/294 [==============================] - 2s 8ms/step - loss: 0.0754 - accuracy: 0.9737 - precision_252: 0.9653 - recall_252: 0.9827 - auc_2: 0.9952
Epoch 5/26
294/294 [==============================] - 2s 8ms/step - loss: 0.0521 - accuracy: 0.9820 - precision_252: 0.9771 - recall_252: 0.9872 - auc_2: 0.9974
Epoch 6/26
294/294 [==============================] - 2s 8ms/step - loss: 0.0356 - accuracy: 0.9876 - precision_252: 0.9845 - recall_252: 0.9909 - auc_2: 0.9986
Epoch 7/26
294/294 [=============

In [ ]:
best_model2.summary()

Model: "sequential_252"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_252 (Bidirect  (None, 104)              131456    
 ional)                                                          
                                                                 
 dropout_453 (Dropout)       (None, 104)               0         
                                                                 
 dense_576 (Dense)           (None, 52)                5460      
                                                                 
 dropout_454 (Dropout)       (None, 52)                0         
                                                                 
 dense_577 (Dense)           (None, 52)                2756      
                                                                 
 dropout_455 (Dropout)       (None, 52)                0         
                                                    

In [ ]:
train_results = best_model2.evaluate(X_train_over, y_train_over)
precision = train_results[2]
recall = train_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Train Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {train_results[1]}\nAUROC:{train_results[4]}')

706/706 [==============================] - 3s 3ms/step - loss: 5.3620e-04 - accuracy: 0.9998 - precision_252: 0.9996 - recall_252: 1.0000 - auc_2: 1.0000
Train Results
Precision: 0.9996459484100342
Recall: 1.0
F1 Score: 0.9998229428613363
Accuracy: 0.9998229146003723
AUROC:1.0000001192092896


In [ ]:
test_results = best_model2.evaluate(X_test, y_test)
precision = test_results[2]
recall = test_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Test Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {test_results[1]}\nAUROC:{test_results[4]}')

90/90 [==============================] - 0s 3ms/step - loss: 0.0643 - accuracy: 0.9892 - precision_252: 0.6984 - recall_252: 0.7857 - auc_2: 0.9342
Test Results
Precision: 0.6984127163887024
Recall: 0.7857142686843872
F1 Score: 0.7394958008531877
Accuracy: 0.9892361164093018
AUROC:0.9341992735862732


#Hyperparameter tuning (Round 3)

Best test F1 score: 0.712

In [ ]:
def bidirectional_lstm_tuner(batch_size, neurons, activation, optimizer, dropout_rate, learning_rate, epochs, layers1, layers2):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                  'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                  'Adagrad':Adagrad(learning_rate=learning_rate),}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)

    def create_model():
      # for reproducibility 
      np.random.seed(42)
      tensorflow.random.set_seed(42)

      tuned_model = Sequential()
      tuned_model.add(Bidirectional(LSTM(neurons, return_sequences=False)))  
      tuned_model.add(Dropout(dropout_rate, seed=42))

      for i in range(layers1): #varying first set of hidden layers
        tuned_model.add(Dense(neurons, activation=activation))
        tuned_model.add(Dropout(dropout_rate, seed=42))

      for i in range(layers2): #varying second set of hidden layers
        tuned_model.add(Dense(neurons, activation=activation))

      tuned_model.add(Dense(1, activation='sigmoid')) #output layer
      # tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])
      tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall()])
      return tuned_model

    es = EarlyStopping(monitor="loss", patience=3)
    mc = ModelCheckpoint('Best_tuned_bidirectional_LSTM_.h5', monitor='val_loss',mode='min',verbose=2, save_best_only=True)

    tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0) 
    score = cross_val_score(tune_rnn, X_train_over, y_train_over, fit_params={'callbacks':[es]}, error_score='raise', scoring='f1').mean() #5 fold cross validation default
    return score

In [ ]:
params_bidirectional_lstm ={
    'neurons': (10, 64),
    'activation':(0, 7),
    'optimizer':(0, 4),
    'learning_rate':(0.0001, 0.001),
    'batch_size':(32, 128),
    'epochs':(15, 30),
    'layers1':(0,3),
    'layers2':(0,3),
    'dropout_rate':(0, 0.4)
}

In [ ]:
# Run Bayesian Optimization
bidirectional_lstm_bo = BayesianOptimization(bidirectional_lstm_tuner, params_bidirectional_lstm, random_state=42)
bidirectional_lstm_bo.maximize(init_points=20, n_iter=5)

|   iter    |  target   | activa... | batch_... | dropou... |  epochs   |  layers1  |  layers2  | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 3ms/step
| 1         | 0.7486    | 2.622     | 123.3     | 0.2928    | 23.98     | 0.4681    | 0.468     | 0.0001523 | 56.77     | 2.404     |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 2ms/step
| 2         | 0.8683    | 4.957     | 33.98     | 0.388     | 27.49     | 0.637     | 0.5455    | 0.0002651 | 26.43     | 2.099     |


...

142/142 [==============================] - 1s 2ms/step
| 23        | 0.5788    | 3.384     | 66.21     | 0.0       | 15.0      | 3.0       | 1.541     | 0.001     | 53.93     | 4.0       |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 2s 2ms/step
| 24        | 0.1495    | 6.772     | 58.87     | 0.0       | 18.47     | 3.0       | 0.0       | 0.001     | 56.43     | 0.0       |


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


142/142 [==============================] - 1s 3ms/step
| 25        | 0.9573    | 2.893     | 63.78     | 0.1625    | 20.62     | 1.133     | 1.524     | 0.0008711 | 53.05     | 1.331     |


In [ ]:
bidirectional_lstm_bo.max['params']

{'activation': 1.3466763722200286,
 'batch_size': 62.057661802744875,
 'dropout_rate': 0.0,
 'epochs': 18.997758809674913,
 'layers1': 3.0,
 'layers2': 0.0,
 'learning_rate': 0.001,
 'neurons': 55.39979522299761,
 'optimizer': 1.2288553110171885}

In [ ]:
def create_model_best_params(params):
  learning_rate = params['learning_rate']
  optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad']
  optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                'Adagrad':Adagrad(learning_rate=learning_rate),}
  activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'relu']
  neurons = round(params['neurons'])
  activation = activationL[round(params['activation'])]
  optimizer = optimizerD[optimizerL[round(params['optimizer'])]]
  batch_size = round(params['batch_size'])
  epochs = round(params['epochs'])
  layers1 = round(params['layers1'])
  layers2 = round(params['layers2'])
  dropout_rate = params['dropout_rate']

  # for reproducibility 
  np.random.seed(42)
  tensorflow.random.set_seed(42)
  tuned_model = Sequential()
  tuned_model.add(Bidirectional(LSTM(neurons, return_sequences=False)))  
  tuned_model.add(Dropout(dropout_rate, seed=42))

  for i in range(layers1): #varying first set of hidden layers
    tuned_model.add(Dense(neurons, activation=activation))
    tuned_model.add(Dropout(dropout_rate, seed=42))
  
  for i in range(layers2): #varying second set of hidden layers
        tuned_model.add(Dense(neurons, activation=activation))

  tuned_model.add(Dense(1, activation='sigmoid')) #output layer
  tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy',metrics.Precision(), metrics.Recall(),metrics.AUC()])
  return tuned_model

In [ ]:
best_params = {'activation': 1.3466763722200286,
              'batch_size': 62.057661802744875,
              'dropout_rate': 0.0,
              'epochs': 18.997758809674913,
              'layers1': 3.0,
              'layers2': 0.0,
              'learning_rate': 0.001,
              'neurons': 55.39979522299761,
              'optimizer': 1.2288553110171885}
best_model3 = create_model_best_params(best_params)
best_model3.fit(X_train_over, y_train_over, epochs=round(best_params['epochs']), batch_size = round(best_params['batch_size']))

Epoch 1/19
365/365 [==============================] - 8s 7ms/step - loss: 0.3421 - accuracy: 0.8350 - precision_378: 0.8133 - recall_378: 0.8696 - auc_3: 0.9278
Epoch 2/19
365/365 [==============================] - 3s 7ms/step - loss: 0.0722 - accuracy: 0.9789 - precision_378: 0.9659 - recall_378: 0.9929 - auc_3: 0.9943
Epoch 3/19
365/365 [==============================] - 3s 7ms/step - loss: 0.0282 - accuracy: 0.9927 - precision_378: 0.9876 - recall_378: 0.9979 - auc_3: 0.9988
Epoch 4/19
365/365 [==============================] - 3s 7ms/step - loss: 0.0161 - accuracy: 0.9960 - precision_378: 0.9932 - recall_378: 0.9988 - auc_3: 0.9993
Epoch 5/19
365/365 [==============================] - 3s 7ms/step - loss: 0.0124 - accuracy: 0.9966 - precision_378: 0.9952 - recall_378: 0.9980 - auc_3: 0.9996
Epoch 6/19
365/365 [==============================] - 2s 7ms/step - loss: 0.0061 - accuracy: 0.9987 - precision_378: 0.9981 - recall_378: 0.9993 - auc_3: 0.9998
Epoch 7/19
365/365 [==============

In [ ]:
best_model3.summary()

Model: "sequential_378"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_378 (Bidirect  (None, 110)              140360    
 ional)                                                          
                                                                 
 dropout_781 (Dropout)       (None, 110)               0         
                                                                 
 dense_1049 (Dense)          (None, 55)                6105      
                                                                 
 dropout_782 (Dropout)       (None, 55)                0         
                                                                 
 dense_1050 (Dense)          (None, 55)                3080      
                                                                 
 dropout_783 (Dropout)       (None, 55)                0         
                                                    

In [ ]:
train_results = best_model3.evaluate(X_train_over, y_train_over)
precision = train_results[2]
recall = train_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Train Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {train_results[1]}\nAUROC:{train_results[4]}')

706/706 [==============================] - 4s 3ms/step - loss: 1.2259e-04 - accuracy: 1.0000 - precision_378: 1.0000 - recall_378: 1.0000 - auc_3: 1.0000
Train Results
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Accuracy: 1.0
AUROC:1.0


In [ ]:
test_results = best_model3.evaluate(X_test, y_test)
precision = test_results[2]
recall = test_results[3]
f1_score = 2 * (precision * recall) / (precision + recall)
print('Test Results')
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}\nAccuracy: {test_results[1]}\nAUROC:{test_results[4]}')

90/90 [==============================] - 0s 3ms/step - loss: 0.0586 - accuracy: 0.9896 - precision_378: 0.7708 - recall_378: 0.6607 - auc_3: 0.8722
Test Results
Precision: 0.7708333134651184
Recall: 0.6607142686843872
F1 Score: 0.7115384431985708
Accuracy: 0.9895833134651184
AUROC:0.8722144961357117
